# Liste des problemes
- Impossible d'importer le module standard "LZMA"

In [5]:
using Pkg
Pkg.activate("venv")
# Pkg.add("LightXML")
# Pkg.add("SeaPearl")
# Pkg.add(name="XML", version="0.2.0")
Pkg.status()

      Status `~/Documents/CORAIL/SeaPearl/XCSP3-Julia-Parser/venv/Project.toml`
  [9c8b4983] LightXML v0.9.0
  [c13076dc] SeaPearl v0.4.3
  [72c71f33] XML v0.2.0


  Activating project at `~/Documents/CORAIL/SeaPearl/XCSP3-Julia-Parser/venv`


# 1. Convert file

In [4]:
include("src/utils/lzma_utils.jl")

file_name = "Haystacks-ext-05_c18.xml"
lzma_root_dir = "instancesXCSP22/lzma/MiniCSP/"
lzma_file_path = lzma_root_dir * file_name * ".lzma"
xml_root_dir = "instancesXCSP22/xml/MiniCSP/"

convert_lzma_file_to_xml(lzma_file_path, xml_root_dir)

LoadError: LoadError: ArgumentError: Package LZMA not found in current path:
- Run `import Pkg; Pkg.add("LZMA")` to install the LZMA package.

in expression starting at /home/bourgeat/Documents/CORAIL/SeaPearl/XCSP3-Julia-Parser/src/utils/lzma_utils.jl:2

# 2. Parsing

In [6]:
using XML

include("src/utils/xml_utils.jl")

file_name2 = "CostasArray-15_mc22.xml"
file_name3 = "NumberPartitioning-20_mc22.xml"
file_name4 = "Haystacks-ext-05_c18.xml"
file_name5 = "Hadamard-17_mc22.xml"
file_name6 = "Eternity-06-06_mc22.xml"
file_name7 = "Crypto1-0-64-0-0x3eb894e67055f622-19_c22.xml"
file_name8 = "Crossword-m18-ogd2008-vg-04-05_c18.xml"
file_name9 = "AircraftLanding-table-airland01_mc22.xml"

file_name = file_name5

xml_root_dir = "instancesXCSP22/xml/MiniCSP/"

file_path = xml_root_dir * file_name

file = open(file_path)
xml_string = read(file, String)
close(file)

# Parser le contenu du fichier XML
doc = parse(xml_string)

# Récupérer le noeud instance 
instance = find_element(doc, "instance")

#Récupérer les noeuds variables et constraints
variables = find_element(instance, "variables")
constraints = find_element(instance, "constraints")

Node Element <constraints> (4 children)

## 2.1. Variables parsing

In [26]:
include("src/parsing/variables.jl")

sort_intervals (generic function with 1 method)

In [7]:
using SeaPearl

abstract type AbstractSeaPearlIntVar{T} end

struct VectorSeaPearlIntVar{T} <: AbstractSeaPearlIntVar{T}
    data::Vector{T}
end

struct MatrixSeaPearlIntVar{T} <: AbstractSeaPearlIntVar{T}
    data::Matrix{T}
end


In [8]:
#Create model
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
dict_var = Dict{String, Any}()

Dict{String, Any}()

In [29]:
for var in children(variables)
    id = attributes(var)["id"]
    if var.tag == "array"
        dict_var[id] = parse_array_variable(var, model, trailer)
    end
end

In [32]:
dict_var["x"][1]

SeaPearl.IntVar: x[0] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20]

## 2.2. Constraints parsing

In [9]:
include("src/parsing/constraints.jl")

get_constraint_variables (generic function with 1 method)

In [10]:
for con in children(constraints)
    parse_constraint(con, dict_var, model, trailer)
    break
end
println(model.constraints)

SeaPearl.Constraint[]


In [11]:
"x[1..2][] y[]"

In [11]:
sum_con = children(constraints)[1]

Node Element <sum> (2 children)

In [ ]:
czdciljzepiocjzepoqcjk

# Parsing Sum constraint

In [ ]:
relations = {lt,le,gt,ge,eq,ne} # lower than, lower or equal, greater than, greater or equal, equal, not equal

In [22]:
sum_con = children(constraints)
for con_node in sum_con
    if con_node.tag == "sum"
        println(con_node)
        for e in children(con_node)
            println("--", e)
            for ee in children(e)
                println("----", ee)
            end
        end
    end
end


Node Element <sum> (2 children)
--Node Element <list> (1 child)
----Node Text "x[]"
--Node Element <condition> (1 child)
----Node Text "(eq,1)"
Node Element <sum> (2 children)
--Node Element <list> (1 child)
----Node Text "y[]"
--Node Element <condition> (1 child)
----Node Text "(eq,1)"


In [46]:
# Comme c'est utilisé dans la fonction parsing peut être qu'il y a une boucle for en trop (celle avant e chack de sum)

function get_relation_sum_constraint(constraint::Node)
    relation_list = [] #String()
    value_list = [] #Array{Int}()

    sum_con = children(constraints)
    for con_node in sum_con
        if con_node.tag == "sum"
            for con_node_2 in children(con_node)
                if con_node_2.tag == "condition"
                    for con_node_3 in children(con_node_2)
                        con_node_3_str = con_node_3.value

                        # test # 
                        con_node_3_str = "(in ,2..5)"
                        # test #
                        
                        relation = match(r"\((\w+),(\d+)\)", con_node_3_str).captures[1]
                        if relation != in
                            value = parse(Int, match(r"\((\w+),(\d+)\)", con_node_3_str).captures[2])
                            value = [value]
                        else
                            min_value, max_value = parse.(Int, split(match(r"\((\w+),(\d+\.\.\d+)\)", con_node_3_str).captures[2], ".."))
                            value = [min_value, max_value]
                        end
                        push!(relation_list, relation)
                        push!(value_list, value)
                    end
                end
            end
        end
    end

    return relation_list, value_list
end

relation, value = get_relation_sum_constraint(constraints)
println(relation)
println(value)

ErrorException: type Nothing has no field captures

In [ ]:
function parse_constraint(constraint::Node, variables::Dict{String, Any}, model::SeaPearl.CPModel, trailer::SeaPearl.Trailer)
    tag = constraint.tag

    if tag == "sum"
        str_constraint_variables = children(constraint)[1].value
        sum_vars = get_constraint_variables(str_constraint_variables, variables)
        relations, value = get_relation_sum_constraint(constraint) # 'value' is an array (one per sum constraints) of an array with 1 value in the general case 
                                                                   # and 2 value in case of relation operand 'in'
        
        if relation == "lt":
            con = con = SeaPearl.SumLessThan(sum_vars, value[1]-1, trailer)
            SeaPearl.addConstraint!(model, con)

        else if relation == "le":
            con = SeaPearl.SumLessThan(sum_vars, value[1], trailer)
            SeaPearl.addConstraint!(model, con)

        else if relation == "gt":
            con = con = SeaPearl.SumGreaterThan(sum_vars, value[1]+1, trailer)
            SeaPearl.addConstraint!(model, con)

        else if relation == "ge":
            con = SeaPearl.SumGreaterThan(sum_vars, value[1], trailer)
            SeaPearl.addConstraint!(model, con)

        else if relation == "eq":
            # add a variable y, add the constraint "y == v" (equal.jl),
            # add the constraint "sum_vars == y"
            y = SeaPearl.IntVar
            con_y = SeaPearl.EqualConstant(y, value[1], trailer)
            con_sum = SeaPearl.SumToVariable(sum_vars, y, trailer)
            SeaPearl.addConstraint!(model, con_y)
            SeaPearl.addConstraint!(model, con_sum)

        else if relation == "ne":
            # add the variable y, add the constraint "y != v" (notequal.jl),
            # add the constraint "sum_vars == y"
            y = SeaPearl.IntVar
            con_y = SeaPearl.NotEqualConstant(y, value[1], trailer)
            con_sum = SeaPearl.SumToVariable(sum_vars, y, trailer)
            SeaPearl.addConstraint!(model, con_y)
            SeaPearl.addConstraint!(model, con_sum)

        else if relation == "in":
            con_greater = SeaPearl.SumGreaterThan(sum_vars, value[1], trailer)
            con_less = SeaPearl.SumLessThan(sum_vars, value[2], trailer)
            SeaPearl.addConstraint!(model, con_greater)
            SeaPearl.addConstraint!(model, con_less)

        else:
            error("Unknown relation")
        end
    end
end

# TODO : penser à faire le cas où on a des groupes de variables au lieu d'une liste